In [1]:
from pandas.tseries.offsets import *
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import itertools
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import grid_search
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
path = {}
path['train'] = {'order':'./training_data/order_data/order_data_{}', 
                    'weather': './training_data/weather_data/weather_data_{}',
                    'traffic': './training_data/traffic_data/traffic_data_{}',
                    'district':'./training_data/cluster_map/cluster_map',
                    'poi':'./training_data/poi_data/poi_data'}
path['test'] = {'order':'./test_data/order_data/order_data_{}_test', 
                'weather': './test_data/weather_data/weather_data_{}_test',
                'traffic': './test_data/traffic_data/traffic_data_{}_test',
                'district':'./test_data/cluster_map/cluster_map'}

M = np.timedelta64(1, 'm') # base time stamp of 1 minute

test_slot1 = range(46,154,12) # The test slot for day 22, 26, 30
test_slot2 = range(58,154,12) # The test slot for day 24, 28

D_range = range(1,67) # List of all district Ids
T_range = range(1,145) # List of all time slots

# Dictionary of District Info Table
district_dict = pd.read_table(path['train']['district'], header=None, index_col=0)
district_dict = district_dict[1].to_dict()

## Utility Funtion

In [3]:
def index(district, slot):
    if type(district) is int:
        if type(slot) is int:
            return [x for x in itertools.product([district],[slot])]
        else:
            return [x for x in itertools.product([district],slot)]
    else:
        if type(slot) is int:
            return [x for x in itertools.product(district,[slot])]
        else:
            return [x for x in itertools.product(district,slot)]

In [4]:
def District(df):
    return df['district'].apply(lambda x: district_dict[x])

In [5]:
def Weekday(df):
    return pd.to_datetime(df['time']).apply(lambda x: x.weekday())

In [6]:
def Time(df, day):
    time = pd.to_datetime(df['time'])
    time = (time - pd.Timestamp(day)) / M / 10 + 1
    return time.astype(int)

In [7]:
def preprocessing(day):
    X = train_order[day].reindex(index(D_range,T_range)).fillna(0)
    X['diff'] = pd.Series(index=X.index)
    for D in D_range:
        X.loc[(D, T_range),'diff'] = X.loc[(D, T_range),'gap'].diff().shift(-1).fillna(0)
    X = pd.concat([X,train_traffic[day]],axis=1).dropna()
    X = X.join(train_weather[day], on='time')
    Y = X['diff']
    X.drop('diff', axis=1, inplace=True)
    return X, Y

## Getting testing data and training data

In [8]:
def Traffic(day, option):
    df = pd.read_table(path[option]['traffic'].format(day.date()), header=None,
                      names=['district', 'LV1', 'LV2', 'LV3', 'LV4','time'])
    df['district'] = District(df)
    df['weekday'] = Weekday(df)
    df['time'] = Time(df, str(day.date()))
    for L in ['LV{}'.format(n) for n in range(1,5)]:
        df[L]=df[L].apply(lambda x: x.split(':')[1]).astype(int)
    index = pd.MultiIndex.from_arrays([df['district'].values, df['time'].values], names=('district', 'time'))
    return pd.DataFrame({'weekday':df['weekday'].values,
                         'district':df['district'].values,
                         'time':df['time'].values,
                         'LV1':df['LV1'].values, 
                         'LV2':df['LV2'].values,
                         'LV3':df['LV3'].values,
                         'LV4':df['LV4'].values,}, index=index).sort_index()

In [9]:
# DTG: District Time Gap
def DTG(day, option):
    df = pd.read_table(path[option]['order'].format(day.date()), header=None, usecols=[1,3,6],
                      names=['driver', 'district', 'time'])
    df = df[df['driver'].isnull()] 
    df['district'] = District(df)
    df['time'] = Time(df, day.date())
    Order = df.groupby(['district', 'time'])
    return pd.DataFrame({'gap':Order.size()})

In [10]:
def Weather(day, option):
    df = pd.read_table(path[option]['weather'].format(day.date()), header=None,
                      names=['time', 'weather', 'temprature', 'pm2.5'])
    df['time'] = Time(df, day.date())
    df = df.drop_duplicates(subset='time')
    DF = pd.DataFrame({'time': T_range}, columns=df.columns)
    DF = DF.set_index('time')
    DF.update(df.set_index('time'))
    return DF.fillna(method='bfill').fillna(method='ffill')

In [11]:
# Dictionary of order data for testing indexed by date, cols = [gap]
test_order = {} 
# Dictionary of traffic data for testing indexed by date, cols = [weekday, district, time, LV1, LV2, LV3, LV4]
test_traffic = {}
# Dictionary of weather data for testing indexed by date, cols = [temperature, weather, pm2.5]
test_weather = {} 
for day in pd.date_range('1/22/2016', periods=5, freq='2D'):
    test_order[str(day.date())] = DTG(day, 'test')
    test_traffic[str(day.date())] = Traffic(day, 'test')
    test_weather[str(day.date())] = Weather(day, 'test')

In [12]:
# Check data
print len(test_order.keys()) == 5
print len(test_traffic.keys()) == 5
print len(test_weather.keys()) == 5
print all(test_order['2016-01-22'].columns.values == np.array(['gap']))
print all(test_traffic['2016-01-22'].columns.values == np.array(['LV1', 'LV2', 'LV3', 'LV4', 'district', 'time', 'weekday']))
print all(test_weather['2016-01-22'].columns.values == np.array(['weather', 'temprature', 'pm2.5']))

True
True
True
True
True
True


In [13]:
# Dictionary of order data for training indexed by date, cols = [gap]
train_order = {}
# Dictionary of traffic data for training indexed by date, cols = [weekday, district, time, LV1, LV2, LV3, LV4]
train_traffic = {}
# Dictionary of weather data for training indexed by date, cols = [temperature, weather, pm2.5]
train_weather = {}
for day in pd.date_range('1/1/2016', periods=21, freq='D'):
    train_order[str(day.date())] = DTG(day, 'train')
    train_traffic[str(day.date())] = Traffic(day, 'train')
    train_weather[str(day.date())] = Weather(day, 'train')

In [14]:
# Check data
print len(train_order.keys()) == 21
print len(train_traffic.keys()) == 21
print len(train_weather.keys()) == 21

True
True
True


## Preparing training data X and Y

In [19]:
X = []
Y = []
for d in pd.date_range('1/1/2016', periods=21, freq='D'):
    tempX, tempY = preprocessing(str(d.date()))
    X.append(tempX)
    Y.append(tempY)
X = pd.concat(X)
X.sort_index(inplace=True)
Y = pd.concat(Y)
Y.sort_index(inplace=True)

In [20]:
print len(X.columns) == 11
print Y.shape[0] == X.shape[0]

True
True


## Developing Zone

In [ ]:
POI = pd.DataFrame()
with open(path['train']['poi'], 'r') as f:
    for line in f:
        POI = pd.concat( [POI, pd.DataFrame([tuple(line.strip().split('\t'))])], ignore_index=True )
POI.rename(columns={0:'district'}, inplace=True)
POI['district'] = District(POI)

## Medthod3 by GradientBoosting

In [126]:
slot1 = range(45,153,12) # Last time slot of test slot for day 22, 26, 30
slot2 = range(57,153,12) # Last time slot of test slot for day 24, 28
S_range = {'2016-01-22':slot1, '2016-01-24':slot2, '2016-01-26':slot1, '2016-01-28':slot2, '2016-01-30':slot1}

In [137]:
def write3(x, day, slot, regr, mode):
    with open('ans3.csv', mode) as f:
        writer = csv.writer(f, delimiter=',')
        for D in D_range:
            for S in slot:
                key = (D,S)
                if key in x.index:
                    gap = x['gap'].loc[key] + regr.predict(x.loc[key].reshape(1, -1))[0]
                else:
                    gap = 1
                gap = 0 if gap < 0 else gap
                writer.writerow([str(D),'{}-{}'.format(day,S+1), '{:.3f}'.format(gap)])

In [23]:
def score3(day, pred):
    ans = test_order[day].reindex(index(D_range, S_range[day])).fillna(0)
    ans = ans['gap'].values
    pred = pred[ans>0]
    ans = ans[ans>0]
    gap = (ans - pred) / ans
    return np.fabs(gap).sum()/ans.shape[0]

### Predict Day22

In [98]:
# Predict Score on this day
day = '2016-01-22'
# all areas mixed: 0.770059708462
params = {'loss': 'huber', 'alpha': 0.9,
          'max_depth': 3, 'learning_rate': 0.1, 'subsample': 1.0}

# Paramters need to be searched
parameters = {'n_estimators': np.arange(100,600,150)}

In [25]:
# Searching best parameters
scoring_function = make_scorer(mean_squared_error, greater_is_better=False)
regr = grid_search.GridSearchCV(GradientBoostingRegressor(**params), 
                                param_grid=parameters, scoring=scoring_function, cv=3)
regr.fit(X, Y)
Regr = regr.best_estimator_

# Regr = GradientBoostingRegressor(**params)
Regr.fit(X, Y)
print Regr

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1,
             loss='huber', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)


In [138]:
# alpha: 0.9 -> 0.1 : 0.775124360324 -> 1.03167874288
# loss: quantile -> ls : 0.775124360324 -> 0.686632655895
# loss: quantile -> huber : 0.775124360324 -> 0.559654872126
# learning_rate: 0.1 -> 0.9 : 0.559654872126 -> 0.584819846794
# max_depth: 3 -> 5 : 0.584819846794 -> 0.624282373776
# n_estimators: 250 -> 400 : 0.584819846794 -> 0.607329151263
# subsample: 1.0 -> 0.5 : 0.559654872126 -> 0.56019559633
x = test_order[day].reindex(index(D_range,S_range[day])).fillna(0)
x = pd.concat([x,test_traffic[day]], axis=1, join_axes=[x.index]).dropna()
x = x.join(test_weather[day], on='time')
write3(x, day, S_range[day], Regr, 'a')

### Predict Day24

In [139]:
day = '2016-01-24'
x = test_order[day].reindex(index(D_range,S_range[day])).fillna(0)
x = pd.concat([x,test_traffic[day]], axis=1, join_axes=[x.index]).dropna()
x = x.join(test_weather[day], on='time')
write3(x, day, S_range[day], Regr, 'a')

### Predict Day26

In [140]:
day = '2016-01-26'
x = test_order[day].reindex(index(D_range,S_range[day])).fillna(0)
x = pd.concat([x,test_traffic[day]], axis=1, join_axes=[x.index]).dropna()
x = x.join(test_weather[day], on='time')
write3(x, day, S_range[day], Regr, 'a')

### Predict Day28

In [141]:
day = '2016-01-28'
x = test_order[day].reindex(index(D_range,S_range[day])).fillna(0)
x = pd.concat([x,test_traffic[day]], axis=1, join_axes=[x.index]).dropna()
x = x.join(test_weather[day], on='time')
write3(x, day, S_range[day], Regr, 'a')

### Predict Day30

In [142]:
day = '2016-01-30'
x = test_order[day].reindex(index(D_range,S_range[day])).fillna(0)
x = pd.concat([x,test_traffic[day]], axis=1, join_axes=[x.index]).dropna()
x = x.join(test_weather[day], on='time')
write3(x, day, S_range[day], Regr, 'a')

In [ ]:
for D in D_range:
    regr = GradientBoostingRegressor(**params)

    regr.fit(X,Y['gap'])
    write3(x, D, day, test_slot1, regr, 'a')

## Naive Method 2 by using interpolation

In [ ]:
slot1 = range(45,153,12)
slot2 = range(57,153,12)
S_range = {'2016-01-22':slot1, '2016-01-24':slot2, '2016-01-26':slot1, '2016-01-28':slot2, '2016-01-30':slot1}
D_range = range(1,67)

In [ ]:
Day_range = {'2016-01-22':pd.date_range('1/2/2016', periods=3, freq='7D'),
             '2016-01-24':pd.date_range('1/3/2016', periods=3, freq='7D'),
             '2016-01-26':pd.date_range('1/5/2016', periods=3, freq='7D'),
             '2016-01-28':pd.date_range('1/7/2016', periods=3, freq='7D'),
             '2016-01-30':pd.date_range('1/9/2016', periods=2, freq='7D')}
def naive_score2(day):
    ans = test_order[day].select(lambda x: x[1] in S_range[day])
    
    
    deltas = []
    for d in Day_range[day]:
        data = train_order[str(d.date())].reindex([x for x in itertools.product(D_range,range(1,145))]).fillna(0)
        data = data.diff().shift(-1)
        data = data.loc[ans.index]
        deltas.append(data)

    delta = deltas[0]
    for i in range(1,len(deltas)):
        delta.add(deltas[i], fill_value=0)
    pred = test_order[day].shift().loc[ans.index].fillna(0)+(delta/len(deltas))
    gap = (ans - pred) / ans

    return gap.abs().sum()/(66*len(S_range[day]))

In [ ]:
print naive_score2('2016-01-22')
print naive_score2('2016-01-24')
print naive_score2('2016-01-26')
print naive_score2('2016-01-28')
print naive_score2('2016-01-30')

In [ ]:
def slope(day):
    base_points = test_order[day].select(lambda x: x[1] in S_range[day])
    deltas = []
    for d in Day_range[day]:
        data = train_order[str(d.date())].reindex([x for x in itertools.product(D_range,range(1,145))]).fillna(0)
        data = data.diff().shift(-1)
        deltas.append(data)
    delta = deltas[0]
    for i in range(1,len(deltas)):
        delta = delta + deltas[i]
    return base_points, delta/len(deltas)

In [ ]:
def write2(day, base_points, slot, delta, mode):
    with open('ans.csv', mode) as f:
        writer = csv.writer(f, delimiter=',')
        for D in range(1,67):
            for S in slot:
                key = (D,S-1)
                if key in base_points.index:
                    gap = base_points['gap'].loc[key] + delta['gap'].loc[key]
                    gap = base_points['gap'].loc[key] if gap < 0 else gap
                else:
                    gap = 0.0
                writer.writerow([D,'{}-{}'.format(day,S), '{:.3f}'.format(gap)])

### Predict Day22

In [ ]:
day = '2016-01-22'
base_points, delta = slope(day)
write2(day, base_points, test_slot1, delta, 'w')

### Predict Day24

In [ ]:
day = '2016-01-24'
base_points, delta = slope(day)
write2(day, base_points, test_slot2, delta, 'a')

### Predict Day26

In [ ]:
day = '2016-01-26'
base_points, delta = slope(day)
write2(day, base_points, test_slot1, delta, 'a')

### Predict Day28

In [ ]:
day = '2016-01-28'
base_points, delta = slope(day)
write2(day, base_points, test_slot2, delta, 'a')

### Predict Day30

In [ ]:
day = '2016-01-30'
base_points, delta = slope(day)
write2(day, base_points, test_slot1, delta, 'a')

## Predict the score of naive method 1

In [ ]:
Day_range = {'2016-01-22':pd.date_range('1/1/2016', periods=3, freq='7D'),
             '2016-01-24':pd.date_range('1/3/2016', periods=3, freq='7D'),
             '2016-01-26':pd.date_range('1/5/2016', periods=3, freq='7D'),
             '2016-01-28':pd.date_range('1/7/2016', periods=3, freq='7D'),
             '2016-01-30':pd.date_range('1/9/2016', periods=2, freq='7D')}
def naive_score1(day):
    ans = test_order[day].select(lambda x: x[1] in S_range[day])
    
    prediction = []
    for d in Day_range[day]:
        data = train_order[str(d.date())]
        data = data.loc[ans.index].fillna(0)
        prediction.append(data)

    pred = prediction[0]
    for i in range(1,len(prediction)):
        pred.add(prediction[i], fill_value=0)
    pred = pred/len(prediction)
    gap = (ans - pred) / ans

    return gap.abs().sum()/(66*len(S_range[day]))

In [ ]:
print naive_score1('2016-01-22')
print naive_score1('2016-01-24')
print naive_score1('2016-01-26')
print naive_score1('2016-01-28')
print naive_score1('2016-01-30')

## Naive Method 1 by using mean

In [ ]:
test_slot1 = range(46,154,12)
test_slot2 = range(58,154,12)

### Predict Day22

In [ ]:
test_day22 = []

In [ ]:
for day in pd.date_range('1/1/2016', periods=3, freq='7D'):
    # Read data
    order = pd.read_table(path['order'].format(day.date()), header=None, usecols=[1,3,6],
                      names=['driver', 'district', 'time'])
    # Select NA for calculating the value of gap
    order = order[order['driver'].isnull()] 
    # Translating hash to id
    order['district'] = order['district'].apply(lambda x: district[x])
    # Translating timestamp to time_id
    order['time'] = pd.to_datetime(order['time'])
    order['time'] = (order['time'] - day) / M / 10 + 1
    order['time'] = order['time'].astype(int)
    # Grouping by district and time
    test_day22.append(order.groupby(['district', 'time']))

In [ ]:
with open('ans.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    for D in range(1,67):
        for S in test_slot1:
            key = (D,S)
            gap = []
            for data in test_day22:
                if key in data.groups:
                    gap.append(data.get_group(key).shape[0])
                else:
                    gap.append(0)
            writer.writerow([D,'2016-01-22-{}'.format(S), '{:.3f}'.format(np.mean(gap))])

### Predict Day24

In [ ]:
test_day24 = []

In [ ]:
for day in pd.date_range('1/3/2016', periods=3, freq='7D'):
    # Read data
    order = pd.read_table(path['order'].format(day.date()), header=None, usecols=[1,3,6],
                      names=['driver', 'district', 'time'])
    # Select NA for calculating the value of gap
    order = order[order['driver'].isnull()] 
    # Translating hash to id
    order['district'] = order['district'].apply(lambda x: district[x])
    # Translating timestamp to time_id
    order['time'] = pd.to_datetime(order['time'])
    order['time'] = (order['time'] - day) / M / 10 + 1
    order['time'] = order['time'].astype(int)
    # Grouping by district and time
    test_day24.append(order.groupby(['district', 'time']))

In [ ]:
with open('ans.csv', 'a') as f:
    writer = csv.writer(f, delimiter=',')
    for D in range(1,67):
        for S in test_slot2:
            key = (D,S)
            gap = []
            for data in test_day24:
                if key in data.groups:
                    gap.append(data.get_group(key).shape[0])
                else:
                    gap.append(0)
            writer.writerow([D,'2016-01-24-{}'.format(S), '{:.3f}'.format(np.mean(gap))])

### Predict Day26

In [ ]:
test_day26 = []

In [ ]:
for day in pd.date_range('1/5/2016', periods=3, freq='7D'):
    # Read data
    order = pd.read_table(path['order'].format(day.date()), header=None, usecols=[1,3,6],
                      names=['driver', 'district', 'time'])
    # Select NA for calculating the value of gap
    order = order[order['driver'].isnull()] 
    # Translating hash to id
    order['district'] = order['district'].apply(lambda x: district[x])
    # Translating timestamp to time_id
    order['time'] = pd.to_datetime(order['time'])
    order['time'] = (order['time'] - day) / M / 10 + 1
    order['time'] = order['time'].astype(int)
    # Grouping by district and time
    test_day26.append(order.groupby(['district', 'time']))

In [ ]:
with open('ans.csv', 'a') as f:
    writer = csv.writer(f, delimiter=',')
    for D in range(1,67):
        for S in test_slot1:
            key = (D,S)
            gap = []
            for data in test_day26:
                if key in data.groups:
                    gap.append(data.get_group(key).shape[0])
                else:
                    gap.append(0)
            writer.writerow([D,'2016-01-26-{}'.format(S), '{:.3f}'.format(np.mean(gap))])

### Predict Day28

In [ ]:
test_day28 = []

In [ ]:
for day in pd.date_range('1/7/2016', periods=3, freq='7D'):
    # Read data
    order = pd.read_table(path['order'].format(day.date()), header=None, usecols=[1,3,6],
                      names=['driver', 'district', 'time'])
    # Select NA for calculating the value of gap
    order = order[order['driver'].isnull()] 
    # Translating hash to id
    order['district'] = order['district'].apply(lambda x: district[x])
    # Translating timestamp to time_id
    order['time'] = pd.to_datetime(order['time'])
    order['time'] = (order['time'] - day) / M / 10 + 1
    order['time'] = order['time'].astype(int)
    # Grouping by district and time
    test_day28.append(order.groupby(['district', 'time']))

In [ ]:
with open('ans.csv', 'a') as f:
    writer = csv.writer(f, delimiter=',')
    for D in range(1,67):
        for S in test_slot2:
            key = (D,S)
            gap = []
            for data in test_day28:
                if key in data.groups:
                    gap.append(data.get_group(key).shape[0])
                else:
                    gap.append(0)
            writer.writerow([D,'2016-01-28-{}'.format(S), '{:.3f}'.format(np.mean(gap))])

### Predict Day30

In [ ]:
test_day30 = []

In [ ]:
for day in pd.date_range('1/9/2016', periods=2, freq='7D'):
    # Read data
    order = pd.read_table(path['order'].format(day.date()), header=None, usecols=[1,3,6],
                      names=['driver', 'district', 'time'])
    # Select NA for calculating the value of gap
    order = order[order['driver'].isnull()] 
    # Translating hash to id
    order['district'] = order['district'].apply(lambda x: district[x])
    # Translating timestamp to time_id
    order['time'] = pd.to_datetime(order['time'])
    order['time'] = (order['time'] - day) / M / 10 + 1
    order['time'] = order['time'].astype(int)
    # Grouping by district and time
    test_day30.append(order.groupby(['district', 'time']))

In [ ]:
with open('ans.csv', 'a') as f:
    writer = csv.writer(f, delimiter=',')
    for D in range(1,67):
        for S in test_slot1:
            key = (D,S)
            gap = []
            for data in test_day30:
                if key in data.groups:
                    gap.append(data.get_group(key).shape[0])
                else:
                    gap.append(0)
            writer.writerow([D,'2016-01-30-{}'.format(S), '{:.3f}'.format(np.mean(gap))])

## Process order

In [ ]:
day = '2016-01-01'
Day = pd.Timestamp(day)

In [ ]:
order = pd.read_table(path['order'].format(day), header=None, usecols=[1,3,6],
                      names=['driver', 'district_id', 'time'])
order = order[order['driver'].isnull()] # Select NA for calculating the value of gap

### Translating district hash to id

In [ ]:
order['district_id'] = order['district_id'].apply(lambda x: district[x])

### Translating timestamp to slot

In [ ]:
order['time'] = pd.to_datetime(order['time'])

In [ ]:
order['time_slot'] = (order['time'] - Day) / M / 10 + 1
order['time_slot'] = order['time_slot'].astype(int)